### Metrics 1 - Fit, Eval and Save for Net3

In [120]:
import os, sys
import copy as copyroot
import pandas as pd
from IPython.display import display
from matplotlib import pyplot as plt
from fastai2.basics import *
from fastai2.vision.all import *
%load_ext autoreload
%autoreload 2

from module.mnist_helpers import build_df, eda_fig_1, build_dls
from module.mnist_models import FeatsNet
from module.mnist_metrics import metrics_df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [121]:
dls_tl = build_dls(target='topleft')
dls_cr = build_dls(target='center')

In [122]:
x, y = dls_tl.one_batch()
x.shape

torch.Size([64, 1, 28, 28])

In [123]:
def build_net3():
    net3 = nn.Sequential(
                nn.Flatten(),
                nn.Linear(in_features=784, out_features=28),
                nn.Linear(in_features=28, out_features=2, bias=False),
                SigmoidRange(-1., 1),
                )
    return net3


In [124]:
net3_tl = Learner(dls_tl, build_net3(), loss_func=MSELossFlat(), )
net3_tl.fit(10)

epoch,train_loss,valid_loss,time
0,0.100172,0.036617,00:01
1,0.069563,0.046325,00:02
2,0.057316,0.032352,00:01
3,0.047391,0.024952,00:01
4,0.040435,0.025795,00:01
5,0.035377,0.022040,00:01
6,0.031452,0.021814,00:01
7,0.028355,0.021301,00:02
8,0.025838,0.020526,00:01
9,0.023746,0.021133,00:01


In [125]:
net3_cr = Learner(dls_cr, build_net3(), loss_func=MSELossFlat(), )
net3_cr.fit(10)

epoch,train_loss,valid_loss,time
0,0.006741,0.005027,00:02
1,0.005105,0.003243,00:01
2,0.004155,0.002714,00:01
3,0.003507,0.002576,00:01
4,0.003070,0.002481,00:01
5,0.002726,0.002309,00:02
6,0.002468,0.002260,00:01
7,0.002259,0.002254,00:01
8,0.002077,0.002292,00:02
9,0.001935,0.002309,00:01


In [126]:
df_tl = metrics_df(net3_tl, 'Net3', '10 fit epochs')
df_tl

,model,details,split,mse,mae,r2
0,Net3,10 fit epochs,valid,0.021133,0.105367,0.157259
1,Net3,10 fit epochs,train,0.014170,0.087649,0.388304


In [127]:
df_cr = metrics_df(net3_cr, 'Net3', '10 fit epochs')
df_cr

,model,details,split,mse,mae,r2
0,Net3,10 fit epochs,valid,0.002309,0.035305,0.535130
1,Net3,10 fit epochs,train,0.001280,0.027434,0.777206


In [128]:
df_cr['target'] = 'center'
df_tl['target'] = 'topleft'

In [129]:
df = pd.concat((df_cr, df_tl), axis=0)
df

,model,details,split,mse,mae,r2,target
0,Net3,10 fit epochs,valid,0.002309,0.035305,0.535130,center
1,Net3,10 fit epochs,train,0.001280,0.027434,0.777206,center
0,Net3,10 fit epochs,valid,0.021133,0.105367,0.157259,topleft
1,Net3,10 fit epochs,train,0.014170,0.087649,0.388304,topleft


In [119]:
dir_fn = 'assets/metrics-dfs/'
df.to_csv(dir_fn + 'metrics1-df-net3.csv', index=False)

#### Pivoting / Displaying Agg's

In [45]:
pd.concat((df_tl, df_cr), axis=1)

,model,details,split,mse,mae,r2,target,model,details,split,mse,mae,r2,target
0,Net3,10 fit epochs,valid,0.019951,0.102220,0.177112,topleft,Net3,10 fit epochs,valid,0.002067,0.033904,0.588672,center
1,Net3,10 fit epochs,train,0.014677,0.089291,0.347828,topleft,Net3,10 fit epochs,train,0.001223,0.026841,0.788758,center


In [43]:
metric_names = ['mse', 'r2']
df.pivot(index='split', columns='target', values=metric_names)

mse                  r2          
target    center   topleft    center   topleft
split                                         
train   0.001223  0.014677  0.788758  0.347828
valid   0.002067  0.019951  0.588672  0.177112

In [39]:
pvt = df.pivot(index='id', columns='target', values=metric_names)
pvt

mse                  r2          
target    center   topleft    center   topleft
id                                            
0       0.002067       NaN  0.588672       NaN
1       0.001223       NaN  0.788758       NaN
2            NaN  0.019951       NaN  0.177112
3            NaN  0.014677       NaN  0.347828

In [53]:
pvt = pvt.swaplevel(0,1,axis=1)
pvt.sort_index(axis=1, level=0)

target    center             topleft          
             mse        r2       mse        r2
id                                            
0       0.002067  0.588672       NaN       NaN
1       0.001223  0.788758       NaN       NaN
2            NaN       NaN  0.019951  0.177112
3            NaN       NaN  0.014677  0.347828

In [42]:
pvt.columns

MultiIndex([('mse',  'center'),
            ('mse', 'topleft'),
            ( 'r2',  'center'),
            ( 'r2', 'topleft')],
           names=[None, 'target'])

##### Notebook name

In [43]:
%%javascript
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "theNotebook = " + "'"+thename+"'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [44]:
print(theNotebook)

Untitled
